# 기본코드

In [ ]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry
        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=3, sticky="ns")
        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=3, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=3, column=1)
        self.voltage_entry.insert(0, "60,90")
        tk.Label(self.root, text="대기 시간 (s):").grid(row=4, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=4, column=1)
        self.wait_entry.insert(0, "300")
        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=5, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=5, column=1)
        self.sampling_entry.insert(0, "5")
        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=6, column=0, columnspan=2)
        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=7, column=0, columnspan=5)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return "ERROR"

    def start_test(self):
        for label in ["Power Supply", "Electronic Load", "온도 센서"]:
            if label not in self.instrument:
                self.log(f"{label}가 연결되지 않았습니다.")
                messagebox.showerror("연결 오류", f"{label}가 연결되지 않았습니다.")
                return
        try:
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            if not voltages:
                raise ValueError("전압 시퀀스를 입력.")
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
        except Exception as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return
        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        self.open_log_file()
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def open_log_file(self):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.csv_file = open(f"result_{timestamp}.csv", "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Time", "Set_V", "Measured_V", "Current", "Power", "Temperature"])
        self.log("CSV 파일 생성 완료")

    def run_test_sequence(self):
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            for v in self.voltages:
                self.log(f"전압 {v}V 설정 및 테스트 시작")
                try:
                    self.write(supply_label, "*RST")
                    time.sleep(1)
                    self.write(supply_label, "AR 1")
                    time.sleep(1)
                    self.write(supply_label, f"VOLT {v}")
                    time.sleep(1)
                    #self.write(supply_label, "CURR 2.0")
                    #time.sleep(1)
                    self.write(supply_label, "TEST") # test가 아니라 outp on으로 해야할 수도?
                    time.sleep(1)
                    #self.write(supply_label, "OUTP ON")  # 이 줄을 반드시 추가해야 실제 출력 시작됨
                    #time.sleep(1)
                except Exception as e:
                    self.log(f"Power Supply 명령 오류: {e}")
                    continue
                try:
                    self.write(load_label, ":MODE CC")  #:FUNC CURR
                    self.write(load_label, ":CURR 2.0")
                    self.write(load_label, ":LOAD ON")
                except Exception as e:
                    self.log(f"Electronic Load 명령 오류: {e}")
                    continue
                start_time = time.time()
                while time.time() - start_time < self.wait_time:
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    try:
                        
                        temperature = float(self.query(temp_label, ":VAL1:VAL?").strip())
                        voltage_meas = float(self.query(load_label, ":MEAS:VOLT?").strip())
                        current_meas = float(self.query(load_label, ":MEAS:CURR?").strip())
                        power_meas = float(self.query(load_label, ":MEAS:POW?").strip())
                        self.csv_writer.writerow([timestamp, v, voltage_meas, current_meas, power_meas, temperature])
                        self.csv_file.flush()
                        self.log(f"[{timestamp}] V={voltage_meas:.2f}, I={current_meas:.2f}, P={power_meas:.2f}, T={temperature:.2f}")
                    except Exception as e:
                        self.log(f"측정 오류: {e}")
                    time.sleep(self.sampling_interval)
                try:
                    self.write(load_label, ":LOAD OFF")
                    #self.write(supply_label, "OUTP OFF")
                    self.write(supply_label, "*RST")
                except Exception as e:
                    self.log(f"테스트 종료 명령 오류: {e}")
                time.sleep(2)
            self.log("모든 테스트 완료")
        except Exception as e:
            self.log(f"시퀀스 실행 중 오류: {e}")
        finally:
            self.is_testing = False
            if self.csv_file:
                self.csv_file.close()
                self.log("CSV 저장 완료")
            for label, inst in self.instrument.items():
                try:
                    if self.connection_type[label] == "VISA":
                        inst.close()
                    elif self.connection_type[label] == "SERIAL":
                        inst.close()
                    elif self.connection_type[label] == "LAN":
                        inst.close()
                except Exception:
                    pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()


# 기본코드에서 수정(장치 하나만 연결해도 가능하도록)

In [7]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry
        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=3, sticky="ns")
        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=3, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=3, column=1)
        self.voltage_entry.insert(0, "60,90")
        tk.Label(self.root, text="대기 시간 (s):").grid(row=4, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=4, column=1)
        self.wait_entry.insert(0, "300")
        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=5, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=5, column=1)
        self.sampling_entry.insert(0, "5")
        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=6, column=0, columnspan=2)
        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=7, column=0, columnspan=5)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return None
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return None

    def start_test(self):
    # 연결된 장치가 최소 1개 이상인지 체크
        if not self.instrument:
            self.log("연결된 장치가 없습니다.")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        # 기존 입력값 체크와 저장
        try:
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            if not voltages:
                raise ValueError("전압 시퀀스를 입력하세요.")
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
        except Exception as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return

        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval

        self.open_log_file()
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def open_log_file(self):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.csv_file = open(f"result_{timestamp}.csv", "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Time", "Set_V", "Measured_V", "Current", "Power", "Temperature", "Supply_v"])
        self.log("CSV 파일 생성 완료")

    def run_test_sequence(self):
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            for v in self.voltages:
                self.log(f"전압 {v}V 설정 및 테스트 시작")
                time.sleep(0.5)
                # Power Supply 명령
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                        time.sleep(1)
                        self.write(supply_label, "AR 1")
                        time.sleep(1)
                        self.write(supply_label, f"VOLT {v}")
                        time.sleep(1)
                        self.write(supply_label, "TEST") # 필요시 OUTP ON 등으로 변경
                        time.sleep(1)
                    except Exception as e:
                        self.log(f"Power Supply 명령 오류: {e}")
                        #continue
                # Electronic Load 명령
                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":MODE CC")
                        self.write(load_label, ":CURR 11.0")
                        self.write(load_label, ":LOAD ON")
                    except Exception as e:
                        self.log(f"Electronic Load 명령 오류: {e}")
                        #continue

                start_time = time.time()
                while time.time() - start_time < self.wait_time:
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    # 초기값
                    supply_v = temperature = voltage_meas = current_meas = power_meas = "N/A"
                    # 연결된 장비만 쿼리
                    try:
                        if supply_label in self.instrument:
                            val = self.query(supply_label, "TDVOLT?")
                            if val is not None:
                                supply_v = float(val)
                        if temp_label in self.instrument:
                            val = self.query(temp_label, ":VAL1:VAL?")
                            if val is not None:
                                temperature = float(val)
                        if load_label in self.instrument:
                            val_v = self.query(load_label, ":MEAS:VOLT?")
                            val_c = self.query(load_label, ":MEAS:CURR?")
                            val_p = self.query(load_label, ":MEAS:POW?")
                            if val_v is not None:
                                voltage_meas = float(val_v)
                            if val_c is not None:
                                current_meas = float(val_c)
                            if val_p is not None:
                                power_meas = float(val_p)
                    except Exception as e:
                        self.log(f"측정 오류: {e}")

                    self.csv_writer.writerow([timestamp, v, voltage_meas, current_meas, power_meas, temperature])
                    self.csv_file.flush()
                    self.log(f"[{timestamp}] V={voltage_meas}, I={current_meas}, P={power_meas}, T={temperature}")

                # 종료 명령 (연결된 장비만)
                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":LOAD OFF")
                    except Exception as e:
                        self.log(f"Electronic Load 종료 명령 오류: {e}")
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                    except Exception as e:
                        self.log(f"Power Supply 종료 명령 오류: {e}")
                time.sleep(2)
            self.log("모든 테스트 완료")
        except Exception as e:
            self.log(f"시퀀스 실행 중 오류: {e}")
        finally:
            self.is_testing = False
            if self.csv_file:
                self.csv_file.close()
                self.log("CSV 저장 완료")
            for label, inst in self.instrument.items():
                try:
                    if self.connection_type[label] == "VISA":
                        inst.close()
                    elif self.connection_type[label] == "SERIAL":
                        inst.close()
                    elif self.connection_type[label] == "LAN":
                        inst.close()
                except Exception:
                    pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()


#  테스트 중지 + 장치 하나만 연결해도 실행

In [19]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="w")
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3, padx=5, pady=5)
            self.device_entries[label] = entry
        
        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=3, sticky="ns", padx=5, pady=5)
        
        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=3, column=1, padx=5, pady=5)
        self.voltage_entry.insert(0, "60,90")
        
        tk.Label(self.root, text="대기 시간 (s):").grid(row=4, column=0, padx=5, pady=5, sticky="w")
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=4, column=1, padx=5, pady=5, sticky="w")
        self.wait_entry.insert(0, "60")
        
        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=5, column=0, padx=5, pady=5, sticky="w")
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=5, column=1, padx=5, pady=5, sticky="w")
        self.sampling_entry.insert(0, "5")
        
        # 테스트 제어 버튼들
        button_frame = tk.Frame(self.root)
        button_frame.grid(row=6, column=0, columnspan=5, pady=10)
        
        self.start_test_btn = tk.Button(button_frame, text="테스트 시작", command=self.start_test)
        self.start_test_btn.pack(side=tk.LEFT, padx=5)
        
        self.stop_test_btn = tk.Button(button_frame, text="테스트 중지", command=self.stop_test, state=tk.DISABLED)
        self.stop_test_btn.pack(side=tk.LEFT, padx=5)
        
        # 연결 상태 표시
        self.status_label = tk.Label(self.root, text="연결 상태: 대기 중", bg="lightgray")
        self.status_label.grid(row=7, column=0, columnspan=5, sticky="ew", padx=5, pady=5)
        
        # 로그 박스
        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=8, column=0, columnspan=5, padx=5, pady=5)
        
        # 스크롤바 추가
        scrollbar = tk.Scrollbar(self.root, command=self.log_box.yview)
        scrollbar.grid(row=8, column=5, sticky="ns", pady=5)
        self.log_box.config(yscrollcommand=scrollbar.set)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
            # 로그가 너무 많이 쌓이면 오래된 것 삭제
            if int(self.log_box.index('end-1c').split('.')[0]) > 1000:
                self.log_box.delete('1.0', '100.0')
        self.root.after(0, append_log)

    def update_status(self, text, color="lightgray"):
        def update():
            self.status_label.config(text=f"연결 상태: {text}", bg=color)
        self.root.after(0, update)

    def connect_all(self):
        self.update_status("연결 중...", "yellow")
        connected_count = 0
        
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                self.log(f"{label} 주소가 입력되지 않았습니다.")
                continue
                
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 5000  # 타임아웃 증가
                    response = inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA) - {response.strip()}")
                    connected_count += 1
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=5)
                    time.sleep(0.5)  # 시리얼 연결 안정화
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                    connected_count += 1
                elif ":" in address and not "::" in address:
                    try:
                        ip, port = address.split(":")
                        sock = socket.create_connection((ip, int(port)), timeout=5)
                        self.instrument[label] = sock
                        self.connection_type[label] = "LAN"
                        self.log(f"{label} 연결 성공 (LAN)")
                        connected_count += 1
                    except ValueError:
                        raise ValueError("IP:Port 형식이 올바르지 않습니다.")
                else:
                    raise ValueError("주소 형식을 인식할 수 없습니다. (USB, COM, IP:Port 형식 지원)")
                    
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")
                # 연결 실패한 경우 정리
                if label in self.instrument:
                    try:
                        self.instrument[label].close()
                    except:
                        pass
                    del self.instrument[label]
                if label in self.connection_type:
                    del self.connection_type[label]
        
        if connected_count == len(self.device_entries):
            self.update_status("모든 장비 연결됨", "lightgreen")
        elif connected_count > 0:
            self.update_status(f"{connected_count}/{len(self.device_entries)} 장비 연결됨", "orange")
        else:
            self.update_status("연결 실패", "lightcoral")

    def write(self, label, command):
        try:
            if label not in self.instrument:
                raise Exception(f"{label}가 연결되지 않음")
                
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())  # \r\n으로 변경
                inst.flush()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")
            raise

    def query(self, label, command):
        try:
            if label not in self.instrument:
                raise Exception(f"{label}가 연결되지 않음")
                
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.flushInput()  # 입력 버퍼 클리어
                inst.write((command + "\n").encode())
                inst.flush()
                time.sleep(0.5)  # 응답 대기 시간 증가
                response = inst.readline().decode().strip()
                return response
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                response = inst.recv(1024).decode().strip()
                return response
                
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return "ERROR"

    def stop_test(self):
        """테스트 중지 함수"""
        self.is_testing = False
        self.log("테스트 중지 요청됨")
        self.start_test_btn.config(state=tk.NORMAL)
        self.stop_test_btn.config(state=tk.DISABLED)

    def start_test(self):
        # 연결된 장비 확인 (하나 이상 연결되어 있으면 진행)
        connected_labels = [label for label in self.device_entries if label in self.instrument]
        if not connected_labels:
            self.log("연결된 장비가 없습니다. 최소 1개 이상 연결하세요.")
            messagebox.showerror("연결 오류", "최소 1개 이상의 장비가 연결되어야 합니다.")
            return

        # 입력값 검증
        try:
            voltage_text = self.voltage_entry.get().strip()
            if not voltage_text:
                raise ValueError("전압 시퀀스를 입력하세요.")
            voltages = [float(v.strip()) for v in voltage_text.split(",") if v.strip()]
            if not voltages:
                raise ValueError("유효한 전압값을 입력하세요.")
            
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
            if sampling_interval >= wait_time:
                raise ValueError("샘플링 간격은 대기 시간보다 작아야 합니다.")
                
        except ValueError as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return
        except Exception as e:
            self.log(f"예상치 못한 오류: {e}")
            messagebox.showerror("오류", str(e))
            return
        
        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        
        if not self.open_log_file():
            return
            
        self.is_testing = True
        self.start_test_btn.config(state=tk.DISABLED)
        self.stop_test_btn.config(state=tk.NORMAL)
        
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def open_log_file(self):
        try:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"result_{timestamp}.csv"
            self.csv_file = open(filename, "w", newline="", encoding="utf-8")
            self.csv_writer = csv.writer(self.csv_file)
            self.csv_writer.writerow(["Time", "Set_V", "Measured_V", "Current", "Power", "Temperature"])
            self.csv_file.flush()
            self.log(f"CSV 파일 생성 완료: {filename}")
            return True
        except Exception as e:
            self.log(f"CSV 파일 생성 오류: {e}")
            messagebox.showerror("파일 오류", f"CSV 파일을 생성할 수 없습니다: {e}")
            return False

    def run_test_sequence(self):
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            
            self.log(f"테스트 시퀀스 시작 - 전압: {self.voltages}V")
            
            for voltage_idx, v in enumerate(self.voltages):
                if not self.is_testing:
                    self.log("테스트가 중지되었습니다.")
                    break
                    
                self.log(f"전압 {v}V 설정 및 테스트 시작 ({voltage_idx+1}/{len(self.voltages)})")
                
                # Power Supply 설정
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                        time.sleep(2)
                        self.write(supply_label, "AR 1")
                        time.sleep(1)
                        self.write(supply_label, f"VOLT {v}")
                        time.sleep(1)
                        #self.write(supply_label, "CURR 2.0")
                        #time.sleep(1)
                        self.write(supply_label, "TEST") # test가 아니라 outp on으로 해야할 수도?
                        time.sleep(1)
                        # self.write(supply_label, "VOLT:LEV:IMM:AMPL " + str(v))  # 표준 SCPI 명령 사용
                        # time.sleep(1)
                        # self.write(supply_label, "CURR:LEV:IMM:AMPL 5.0")  # 전류 제한 설정
                        # time.sleep(1)
                        # self.write(supply_label, "OUTP ON")  # 출력 켜기
                        # time.sleep(2)
                        self.log(f"Power Supply 설정 완료: {v}V")
                    except Exception as e:
                        self.log(f"Power Supply 설정 오류: {e}")
                        continue
                else:
                    self.log("Power Supply 미연결: 해당 단계 건너뜀")

                # Electronic Load 설정
                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":MODE CC")
                        time.sleep(0.5)
                        self.write(load_label, ":CURR 2.0")
                        time.sleep(0.5)
                        self.write(load_label, ":LOAD ON")
                        time.sleep(1)
                        self.log("Electronic Load 설정 완료: CC 2.0A")
                    except Exception as e:
                        self.log(f"Electronic Load 설정 오류: {e}")
                        continue
                else:
                    self.log("Electronic Load 미연결: 해당 단계 건너뜀")
                
                # 데이터 수집 시작
                start_time = time.time()
                sample_count = 0
                
                while (time.time() - start_time < self.wait_time) and self.is_testing:
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    
                    try:
                        # 측정값 읽기
                        if load_label in self.instrument:
                            try:
                                voltage_meas = float(self.query(load_label, ":MEAS:VOLT?"))
                            except Exception as e:
                                voltage_meas = 0.0
                                self.log(f"VOLT 측정 오류: {e}")
                            try:
                                current_meas = float(self.query(load_label, ":MEAS:CURR?"))
                            except Exception as e:
                                current_meas = 0.0
                                self.log(f"CURR 측정 오류: {e}")
                            try:
                                power_meas = float(self.query(load_label, ":MEAS:POW?"))
                            except Exception as e:
                                power_meas = 0.0
                                self.log(f"POW 측정 오류: {e}")
                        else:
                            voltage_meas = 0.0
                            current_meas = 0.0
                            power_meas = 0.0

                        # 온도 센서 읽기
                        if temp_label in self.instrument:
                            temp_response = self.query(temp_label, ":VAL1:VAL?")
                            try:
                                temperature = float(temp_response)
                            except:
                                temperature = 0.0  # 온도 읽기 실패시 기본값
                                self.log(f"온도 읽기 실패: {temp_response}")
                        else:
                            temperature = 0.0

                        # CSV에 데이터 저장
                        self.csv_writer.writerow([timestamp, v, voltage_meas, current_meas, power_meas, temperature])
                        self.csv_file.flush()
                        
                        sample_count += 1
                        elapsed_time = int(time.time() - start_time)
                        remaining_time = int(self.wait_time - elapsed_time)
                        
                        self.log(f"[{sample_count:03d}] V={voltage_meas:.2f}V, I={current_meas:.2f}A, P={power_meas:.2f}W, T={temperature:.1f}°C (남은시간: {remaining_time}s)")
                        
                    except Exception as e:
                        self.log(f"측정 오류: {e}")
                    
                    time.sleep(self.sampling_interval)
                
                # 각 전압 단계 완료 후 정리
                if load_label in self.instrument or supply_label in self.instrument:
                    try:
                        if load_label in self.instrument:
                            self.write(load_label, ":LOAD OFF")
                            time.sleep(1)
                        if supply_label in self.instrument:
                            self.write(supply_label, "*RST")
                            time.sleep(2)
                        self.log(f"전압 {v}V 테스트 완료")
                    except Exception as e:
                        self.log(f"테스트 종료 명령 오류: {e}")
                else:
                    self.log("테스트 종료 명령: 연결된 장비 없음")
                
                # 다음 전압으로 넘어가기 전 대기
                if voltage_idx < len(self.voltages) - 1:
                    time.sleep(5)
            
            if self.is_testing:
                self.log("모든 전압 시퀀스 테스트 완료!")
            else:
                self.log("테스트가 사용자에 의해 중지되었습니다.")
                
        except Exception as e:
            self.log(f"시퀀스 실행 중 심각한 오류: {e}")
            
        finally:
            self.is_testing = False
            
            # 모든 장비 출력 끄기
            try:
                if "Electronic Load" in self.instrument:
                    self.write("Electronic Load", ":LOAD OFF")
                if "Power Supply" in self.instrument:
                    # self.write("Power Supply", "OUTP OFF")
                    self.write("Power Supply", "*RST")
            except Exception as e:
                self.log(f"장비 종료 명령 오류: {e}")
            
            # CSV 파일 닫기
            if self.csv_file:
                try:
                    self.csv_file.close()
                    self.log("CSV 파일 저장 완료")
                except Exception as e:
                    self.log(f"CSV 파일 저장 오류: {e}")
            
            # GUI 버튼 상태 복원
            self.root.after(0, lambda: (
                self.start_test_btn.config(state=tk.NORMAL),
                self.stop_test_btn.config(state=tk.DISABLED)
            ))
    
    def __del__(self):
        try:
            if hasattr(self, 'csv_file') and self.csv_file:
                self.csv_file.close()
            
            if hasattr(self, 'instrument'):
                for label, inst in self.instrument.items():
                    try:
                        if self.connection_type[label] == "VISA":
                            inst.close()
                        elif self.connection_type[label] == "SERIAL":
                            inst.close()
                        elif self.connection_type[label] == "LAN":
                            inst.close()
                    except Exception:
                        pass
            
            if hasattr(self, 'rm'):
                self.rm.close()
        except Exception:
            pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()

# 기본코드 + 정지 

In [20]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry

        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=3, sticky="ns")

        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=3, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=3, column=1)
        self.voltage_entry.insert(0, "60,90")

        tk.Label(self.root, text="대기 시간 (s):").grid(row=4, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=4, column=1)
        self.wait_entry.insert(0, "300")

        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=5, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=5, column=1)
        self.sampling_entry.insert(0, "5")

        tk.Label(self.root, text="부하 전류 (A):").grid(row=4, column=2)
        self.current_entry = tk.Entry(self.root, width=10)
        self.current_entry.grid(row=4, column=3)
        self.current_entry.insert(0, "11.0")

        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=6, column=0, columnspan=2)

        self.stop_test_btn = tk.Button(self.root, text="테스트 정지", command=self.stop_test)
        self.stop_test_btn.grid(row=6, column=2, columnspan=2)

        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=7, column=0, columnspan=5)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return None
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return None

    def start_test(self):
        if not self.instrument:
            self.log("연결된 장치가 없습니다.")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        try:
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            if not voltages:
                raise ValueError("전압 시퀀스를 입력하세요.")
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
            load_current = float(self.current_entry.get())
            if load_current <= 0:
                raise ValueError("부하 전류는 0보다 커야 합니다.")
        except Exception as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return

        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        self.load_current = load_current

        self.open_log_file()
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def stop_test(self):
        self.is_testing = False
        self.log("테스트 중지 요청됨.")

    def open_log_file(self):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.csv_file = open(f"result_{timestamp}.csv", "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Time", "Set_V", "Measured_V", "Current", "Power", "Temperature", "Supply_v"])
        self.log("CSV 파일 생성 완료")

    def run_test_sequence(self):
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            for v in self.voltages:
                if not self.is_testing:
                    break
                self.log(f"전압 {v}V 설정 및 테스트 시작")
                time.sleep(0.5)
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                        time.sleep(1)
                        self.write(supply_label, "AR 1")
                        time.sleep(1)
                        self.write(supply_label, f"VOLT {v}")
                        time.sleep(1)
                        self.write(supply_label, "TEST")
                        time.sleep(1)
                    except Exception as e:
                        self.log(f"Power Supply 명령 오류: {e}")

                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":MODE CC")
                        self.write(load_label, f":CURR {self.load_current}")
                        self.write(load_label, ":LOAD ON")
                    except Exception as e:
                        self.log(f"Electronic Load 명령 오류: {e}")

                start_time = time.time()
                while time.time() - start_time < self.wait_time:
                    if not self.is_testing:
                        break
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    supply_v = temperature = voltage_meas = current_meas = power_meas = "N/A"
                    try:
                        if supply_label in self.instrument:
                            val = self.query(supply_label, "TDVOLT?")
                            if val is not None:
                                supply_v = float(val)
                        if temp_label in self.instrument:
                            val = self.query(temp_label, ":VAL1:VAL?")
                            if val is not None:
                                temperature = float(val)
                        if load_label in self.instrument:
                            val_v = self.query(load_label, ":MEAS:VOLT?")
                            val_c = self.query(load_label, ":MEAS:CURR?")
                            val_p = self.query(load_label, ":MEAS:POW?")
                            if val_v is not None:
                                voltage_meas = float(val_v)
                            if val_c is not None:
                                current_meas = float(val_c)
                            if val_p is not None:
                                power_meas = float(val_p)
                    except Exception as e:
                        self.log(f"측정 오류: {e}")

                    self.csv_writer.writerow([timestamp, v, voltage_meas, current_meas, power_meas, temperature, supply_v])
                    self.csv_file.flush()
                    self.log(f"[{timestamp}] V={voltage_meas}, I={current_meas}, P={power_meas}, T={temperature}")
                    time.sleep(self.sampling_interval)

                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":LOAD OFF")
                    except Exception as e:
                        self.log(f"Electronic Load 종료 명령 오류: {e}")
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                    except Exception as e:
                        self.log(f"Power Supply 종료 명령 오류: {e}")
                time.sleep(2)
            self.log("모든 테스트 완료")
        except Exception as e:
            self.log(f"시퀀스 실행 중 오류: {e}")
        finally:
            self.is_testing = False
            if self.csv_file:
                self.csv_file.close()
                self.log("CSV 저장 완료")
            for label, inst in self.instrument.items():
                try:
                    if self.connection_type[label] == "VISA":
                        inst.close()
                    elif self.connection_type[label] == "SERIAL":
                        inst.close()
                    elif self.connection_type[label] == "LAN":
                        inst.close()
                except Exception:
                    pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()


# 여러가지 추가한거 문제 있을 수 있음

In [8]:
import tkinter as tk
from tkinter import messagebox, ttk
import pyvisa
import serial
import socket
import threading
import csv
import time
import atexit
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        
        # 상태 변수들
        self.current_voltage = 0
        self.test_progress = 0
        
        # 프로그램 종료 시 정리 작업 등록
        atexit.register(self.cleanup_resources)
        self.root.protocol("WM_DELETE_WINDOW", self.on_closing)
        
        self.build_gui()

    def build_gui(self):
        # 메인 프레임
        main_frame = ttk.Frame(self.root, padding="10")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # 장비 연결 섹션
        connection_frame = ttk.LabelFrame(main_frame, text="장비 연결", padding="5")
        connection_frame.grid(row=0, column=0, columnspan=5, sticky=(tk.W, tk.E), pady=5)
        
        labels = ["Power Supply", "Electronic Load", "온도 센서"]
        self.device_entries = {}
        self.connection_status = {}
        
        for i, label in enumerate(labels):
            ttk.Label(connection_frame, text=label).grid(row=i, column=0, sticky=tk.W, padx=5)
            entry = ttk.Entry(connection_frame, width=40)
            entry.grid(row=i, column=1, columnspan=2, sticky=(tk.W, tk.E), padx=5)
            self.device_entries[label] = entry
            
            # 연결 상태 표시
            status_label = ttk.Label(connection_frame, text="미연결", foreground="red")
            status_label.grid(row=i, column=3, padx=5)
            self.connection_status[label] = status_label

        self.connect_btn = ttk.Button(connection_frame, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=3, padx=5)

        # 테스트 설정 섹션
        settings_frame = ttk.LabelFrame(main_frame, text="테스트 설정", padding="5")
        settings_frame.grid(row=1, column=0, columnspan=5, sticky=(tk.W, tk.E), pady=5)

        # 첫 번째 행
        ttk.Label(settings_frame, text="전압 시퀀스 (V):").grid(row=0, column=0, sticky=tk.W, padx=5)
        self.voltage_entry = ttk.Entry(settings_frame, width=20)
        self.voltage_entry.grid(row=0, column=1, padx=5)
        self.voltage_entry.insert(0, "60,90")

        ttk.Label(settings_frame, text="부하 전류 (A):").grid(row=0, column=2, sticky=tk.W, padx=5)
        self.current_entry = ttk.Entry(settings_frame, width=10)
        self.current_entry.grid(row=0, column=3, padx=5)
        self.current_entry.insert(0, "11.0")

        # 두 번째 행
        ttk.Label(settings_frame, text="대기 시간 (s):").grid(row=1, column=0, sticky=tk.W, padx=5)
        self.wait_entry = ttk.Entry(settings_frame, width=10)
        self.wait_entry.grid(row=1, column=1, padx=5)
        self.wait_entry.insert(0, "5")

        ttk.Label(settings_frame, text="샘플링 간격 (s):").grid(row=1, column=2, sticky=tk.W, padx=5)
        self.sampling_entry = ttk.Entry(settings_frame, width=10)
        self.sampling_entry.grid(row=1, column=3, padx=5)
        self.sampling_entry.insert(0, "5")

        # 컨트롤 섹션
        control_frame = ttk.Frame(main_frame)
        control_frame.grid(row=2, column=0, columnspan=5, pady=10)

        self.start_test_btn = ttk.Button(control_frame, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=0, column=0, padx=5)

        self.stop_test_btn = ttk.Button(control_frame, text="테스트 정지", command=self.stop_test, state="disabled")
        self.stop_test_btn.grid(row=0, column=1, padx=5)

        # 진행률 바
        self.progress_var = tk.StringVar(value="대기 중")
        ttk.Label(control_frame, textvariable=self.progress_var).grid(row=0, column=2, padx=10)
        
        self.progress_bar = ttk.Progressbar(control_frame, mode='indeterminate')
        self.progress_bar.grid(row=0, column=3, padx=5)

        # 실시간 측정값 표시
        status_frame = ttk.LabelFrame(main_frame, text="실시간 측정값", padding="5")
        status_frame.grid(row=3, column=0, columnspan=5, sticky=(tk.W, tk.E), pady=5)

        self.measurement_vars = {
            "전압": tk.StringVar(value="N/A"),
            "전류": tk.StringVar(value="N/A"),
            "전력": tk.StringVar(value="N/A"),
            "온도": tk.StringVar(value="N/A")
        }

        col = 0
        for label, var in self.measurement_vars.items():
            ttk.Label(status_frame, text=f"{label}:").grid(row=0, column=col*2, sticky=tk.W, padx=5)
            ttk.Label(status_frame, textvariable=var, font=('TkDefaultFont', 9, 'bold')).grid(row=0, column=col*2+1, sticky=tk.W, padx=5)
            col += 1

        # 로그 섹션
        log_frame = ttk.LabelFrame(main_frame, text="로그", padding="5")
        log_frame.grid(row=4, column=0, columnspan=5, sticky=(tk.W, tk.E, tk.N, tk.S), pady=5)

        # 스크롤바가 있는 텍스트 위젯
        self.log_box = tk.Text(log_frame, height=15, width=100)
        scrollbar = ttk.Scrollbar(log_frame, orient="vertical", command=self.log_box.yview)
        self.log_box.configure(yscrollcommand=scrollbar.set)
        
        self.log_box.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        scrollbar.grid(row=0, column=1, sticky=(tk.N, tk.S))

        # 그리드 가중치 설정
        main_frame.columnconfigure(0, weight=1)
        main_frame.rowconfigure(4, weight=1)
        log_frame.columnconfigure(0, weight=1)
        log_frame.rowconfigure(0, weight=1)

    def log(self, text, level="INFO"):
        """로그 메시지 추가 (레벨별 색상 구분)"""
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            message = f"[{timestamp}] {text}\n"
            
            # 로그 레벨에 따른 색상 설정
            if level == "ERROR":
                self.log_box.insert(tk.END, message, "error")
                self.log_box.tag_config("error", foreground="red")
            elif level == "WARNING":
                self.log_box.insert(tk.END, message, "warning")
                self.log_box.tag_config("warning", foreground="orange")
            elif level == "SUCCESS":
                self.log_box.insert(tk.END, message, "success")
                self.log_box.tag_config("success", foreground="green")
            else:
                self.log_box.insert(tk.END, message)
            
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def update_connection_status(self, label, connected):
        """연결 상태 업데이트"""
        if connected:
            self.connection_status[label].config(text="연결됨", foreground="green")
        else:
            self.connection_status[label].config(text="미연결", foreground="red")

    def connect_all(self):
        """모든 장비 연결"""
        self.connect_btn.config(state="disabled")
        
        # 기존 연결 해제
        self.disconnect_all()
        
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
                
            try:
                success = False
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    response = inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA): {response.strip()}", "SUCCESS")
                    success = True
                    
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)", "SUCCESS")
                    success = True
                    
                elif ":" in address and not "::" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)", "SUCCESS")
                    success = True
                    
                else:
                    raise ValueError("지원되지 않는 주소 형식입니다.")
                
                self.update_connection_status(label, success)
                    
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}", "ERROR")
                self.update_connection_status(label, False)
        
        self.connect_btn.config(state="normal")

    def disconnect_all(self):
        """모든 장비 연결 해제"""
        for label in list(self.instrument.keys()):
            try:
                inst = self.instrument[label]
                conn_type = self.connection_type[label]
                
                if conn_type == "VISA":
                    inst.close()
                elif conn_type == "SERIAL":
                    inst.close()
                elif conn_type == "LAN":
                    inst.close()
                    
                del self.instrument[label]
                del self.connection_type[label]
                self.update_connection_status(label, False)
                
            except Exception as e:
                self.log(f"{label} 연결 해제 중 오류: {e}", "WARNING")

    def safe_write(self, label, command):
        """안전한 명령 전송"""
        try:
            if label not in self.connection_type or label not in self.instrument:
                return False
                
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
            return True
            
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}", "ERROR")
            self.update_connection_status(label, False)
            return False

    def safe_query(self, label, command, timeout_retry=1):
        """안전한 쿼리 (재시도 포함)"""
        for attempt in range(timeout_retry + 1):
            try:
                if label not in self.connection_type or label not in self.instrument:
                    return None
                    
                conn_type = self.connection_type[label]
                inst = self.instrument[label]
                
                if conn_type == "VISA":
                    result = inst.query(command).strip()
                elif conn_type == "SERIAL":
                    inst.write((command + "\n").encode())
                    time.sleep(0.3)
                    result = inst.readline().decode().strip()
                elif conn_type == "LAN":
                    inst.sendall((command + "\n").encode())
                    result = inst.recv(1024).decode().strip()
                
                # 결과 검증
                if result and result != "":
                    return result
                    
            except Exception as e:
                if attempt == timeout_retry:
                    self.log(f"{label} 쿼리 오류 (최종 실패): {e}", "ERROR")
                    self.update_connection_status(label, False)
                else:
                    time.sleep(0.5)  # 재시도 전 잠시 대기
                    
        return None

    def validate_measurement(self, value, min_val=None, max_val=None):
        """측정값 유효성 검증"""
        try:
            float_val = float(value)
            if min_val is not None and float_val < min_val:
                return False
            if max_val is not None and float_val > max_val:
                return False
            return True
        except:
            return False

    def update_measurements(self, voltage, current, power, temperature):
        """실시간 측정값 업데이트"""
        def update():
            self.measurement_vars["전압"].set(f"{voltage} V" if voltage != "N/A" else "N/A")
            self.measurement_vars["전류"].set(f"{current} A" if current != "N/A" else "N/A")
            self.measurement_vars["전력"].set(f"{power} W" if power != "N/A" else "N/A")
            self.measurement_vars["온도"].set(f"{temperature} °C" if temperature != "N/A" else "N/A")
        self.root.after(0, update)

    def start_test(self):
        """테스트 시작"""
        if not self.instrument:
            self.log("연결된 장치가 없습니다.", "ERROR")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        try:
            # 입력값 검증
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            if not voltages:
                raise ValueError("전압 시퀀스를 입력하세요.")
            
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            load_current = float(self.current_entry.get())
            
            # 범위 검증
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
            if load_current <= 0:
                raise ValueError("부하 전류는 0보다 커야 합니다.")
            if any(v <= 0 for v in voltages):
                raise ValueError("전압값은 0보다 커야 합니다.")
                
        except Exception as e:
            self.log(f"입력값 오류: {e}", "ERROR")
            messagebox.showerror("입력값 오류", str(e))
            return

        # 테스트 매개변수 저장
        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        self.load_current = load_current

        # UI 상태 변경
        self.start_test_btn.config(state="disabled")
        self.stop_test_btn.config(state="normal")
        self.progress_bar.start()
        self.progress_var.set("테스트 준비 중...")

        # CSV 파일 생성
        if not self.open_log_file():
            return

        # 테스트 시작
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def stop_test(self):
        """테스트 중지"""
        self.is_testing = False
        self.log("테스트 중지 요청됨...", "WARNING")
        self.progress_var.set("테스트 중지 중...")

    def open_log_file(self):
        """CSV 로그 파일 생성"""
        try:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            self.csv_file = open(f"result_{timestamp}.csv", "w", newline="", encoding='utf-8-sig')
            self.csv_writer = csv.writer(self.csv_file)
            self.csv_writer.writerow([
                "Time", "Set_Voltage_V", "Measured_Voltage_V", "Current_A", 
                "Power_W", "Temperature_C", "Supply_Voltage_V"
            ])
            self.csv_file.flush()
            self.log("CSV 파일 생성 완료", "SUCCESS")
            return True
        except Exception as e:
            self.log(f"CSV 파일 생성 실패: {e}", "ERROR")
            messagebox.showerror("파일 오류", f"CSV 파일을 생성할 수 없습니다: {e}")
            return False

    def run_test_sequence(self):
        """테스트 시퀀스 실행"""
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            
            total_steps = len(self.voltages)
            
            for step, voltage in enumerate(self.voltages):
                if not self.is_testing:
                    break
                
                self.current_voltage = voltage
                self.progress_var.set(f"전압 {voltage}V 테스트 ({step+1}/{total_steps})")
                self.log(f"=== 전압 {voltage}V 설정 및 테스트 시작 ===")
                
                # Power Supply 설정
                if supply_label in self.instrument:
                    self.setup_power_supply(supply_label, voltage)
                
                # Electronic Load 설정
                if load_label in self.instrument:
                    self.setup_electronic_load(load_label)
                
                # 측정 루프
                self.measurement_loop(step+1, total_steps, supply_label, load_label, temp_label)
                
                # 장비 종료
                self.shutdown_equipment(supply_label, load_label)
                
                if self.is_testing and step < total_steps - 1:
                    self.log("다음 전압 설정 전 대기...")
                    time.sleep(2)
            
            if self.is_testing:
                self.log("=== 모든 테스트 완료 ===", "SUCCESS")
            else:
                self.log("=== 테스트가 사용자에 의해 중지됨 ===", "WARNING")
                
        except Exception as e:
            self.log(f"시퀀스 실행 중 치명적 오류: {e}", "ERROR")
        finally:
            self.cleanup_test()

    def setup_power_supply(self, label, voltage):
        """Power Supply 설정"""
        try:
            commands = [
                ("*RST", 1),
                ("AR 1", 1),
                (f"VOLT {voltage}", 1),
                ("TEST", 1)
            ]
            
            for cmd, delay in commands:
                if not self.is_testing:
                    break
                if not self.safe_write(label, cmd):
                    raise Exception(f"명령 전송 실패: {cmd}")
                time.sleep(delay)
                
            self.log(f"Power Supply 설정 완료: {voltage}V", "SUCCESS")
            
        except Exception as e:
            self.log(f"Power Supply 설정 오류: {e}", "ERROR")
            raise

    def setup_electronic_load(self, label):
        """Electronic Load 설정"""
        try:
            commands = [
                ":MODE CC",
                f":CURR {self.load_current}",
                ":LOAD ON"
            ]
            
            for cmd in commands:
                if not self.is_testing:
                    break
                if not self.safe_write(label, cmd):
                    raise Exception(f"명령 전송 실패: {cmd}")
                time.sleep(0.5)
                
            self.log(f"Electronic Load 설정 완료: {self.load_current}A", "SUCCESS")
            
        except Exception as e:
            self.log(f"Electronic Load 설정 오류: {e}", "ERROR")
            raise

    def measurement_loop(self, current_step, total_steps, supply_label, load_label, temp_label):
        """측정 루프"""
        start_time = time.time()
        measurement_count = 0
        
        while time.time() - start_time < self.wait_time and self.is_testing:
            elapsed_time = time.time() - start_time
            remaining_time = self.wait_time - elapsed_time
            
            self.progress_var.set(f"측정 중... ({current_step}/{total_steps}) - 남은시간: {int(remaining_time)}s")
            
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # 측정값 초기화
            measurements = {
                'supply_v': "N/A",
                'temperature': "N/A",
                'voltage_meas': "N/A",
                'current_meas': "N/A",
                'power_meas': "N/A"
            }
            
            # 각 장비에서 측정
            self.measure_from_supply(supply_label, measurements)
            self.measure_from_load(load_label, measurements)
            self.measure_from_temp_sensor(temp_label, measurements)
            
            # CSV에 기록
            if self.csv_writer:
                self.csv_writer.writerow([
                    timestamp, self.current_voltage,
                    measurements['voltage_meas'], measurements['current_meas'],
                    measurements['power_meas'], measurements['temperature'],
                    measurements['supply_v']
                ])
                self.csv_file.flush()
            
            # 실시간 표시 업데이트
            self.update_measurements(
                measurements['voltage_meas'], measurements['current_meas'],
                measurements['power_meas'], measurements['temperature']
            )
            
            # 로그 출력
            measurement_count += 1
            self.log(f"측정#{measurement_count}: V={measurements['voltage_meas']}, "
                    f"I={measurements['current_meas']}, P={measurements['power_meas']}, "
                    f"T={measurements['temperature']}")
            
            time.sleep(self.sampling_interval)

    def measure_from_supply(self, label, measurements):
        """Power Supply에서 측정"""
        if label in self.instrument:
            try:
                val = self.safe_query(label, "TDVOLT?")
                if val and self.validate_measurement(val, 0, 1000):
                    measurements['supply_v'] = float(val)
            except Exception as e:
                self.log(f"Power Supply 측정 오류: {e}", "WARNING")

    def measure_from_load(self, label, measurements):
        """Electronic Load에서 측정"""
        if label in self.instrument:
            try:
                voltage = self.safe_query(label, ":MEAS:VOLT?")
                current = self.safe_query(label, ":MEAS:CURR?")
                power = self.safe_query(label, ":MEAS:POW?")
                
                if voltage and self.validate_measurement(voltage, 0, 1000):
                    measurements['voltage_meas'] = float(voltage)
                if current and self.validate_measurement(current, 0, 100):
                    measurements['current_meas'] = float(current)
                if power and self.validate_measurement(power, 0, 10000):
                    measurements['power_meas'] = float(power)
                    
            except Exception as e:
                self.log(f"Electronic Load 측정 오류: {e}", "WARNING")

    def measure_from_temp_sensor(self, label, measurements):
        """온도 센서에서 측정"""
        if label in self.instrument:
            try:
                val = self.safe_query(label, ":VAL1:VAL?")
                if val and self.validate_measurement(val, -50, 200):
                    measurements['temperature'] = float(val)
            except Exception as e:
                self.log(f"온도 센서 측정 오류: {e}", "WARNING")

    def shutdown_equipment(self, supply_label, load_label):
        """장비 종료"""
        # Electronic Load 먼저 끄기
        if load_label in self.instrument:
            try:
                self.safe_write(load_label, ":LOAD OFF")
                self.log("Electronic Load 종료 완료")
            except Exception as e:
                self.log(f"Electronic Load 종료 오류: {e}", "WARNING")
        
        # Power Supply 리셋
        if supply_label in self.instrument:
            try:
                self.safe_write(supply_label, "*RST")
                self.log("Power Supply 리셋 완료")
            except Exception as e:
                self.log(f"Power Supply 리셋 오류: {e}", "WARNING")

    def cleanup_test(self):
        """테스트 종료 후 정리"""
        def cleanup_ui():
            self.is_testing = False
            self.start_test_btn.config(state="normal")
            self.stop_test_btn.config(state="disabled")
            self.progress_bar.stop()
            self.progress_var.set("테스트 완료")
            
            # 측정값 초기화
            for var in self.measurement_vars.values():
                var.set("N/A")
        
        self.root.after(0, cleanup_ui)
        
        # CSV 파일 닫기
        if self.csv_file:
            try:
                self.csv_file.close()
                self.csv_file = None
                self.log("CSV 파일 저장 완료", "SUCCESS")
            except Exception as e:
                self.log(f"CSV 파일 저장 오류: {e}", "ERROR")

    def cleanup_resources(self):
        """리소스 정리"""
        self.is_testing = False
        
        if self.csv_file:
            try:
                self.csv_file.close()
            except:
                pass
        
        self.disconnect_all()
        
        try:
            self.rm.close()
        except:
            pass

    def on_closing(self):
        """프로그램 종료 시 처리"""
        if self.is_testing:
            result = messagebox.askyesno("종료 확인", "테스트가 진행 중입니다. 정말로 종료하시겠습니까?")
            if not result:
                return
        
        self.cleanup_resources()
        self.root.destroy()

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()

# 파워미터로 값 + 로드 채널 1개 경우 

In [8]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서", "Power Meter"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry

        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=4, sticky="ns")

        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=4, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=4, column=1)
        self.voltage_entry.insert(0, "60,90")

        tk.Label(self.root, text="대기 시간 (s):").grid(row=5, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=5, column=1)
        self.wait_entry.insert(0, "300")

        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=6, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=6, column=1)
        self.sampling_entry.insert(0, "5")

        tk.Label(self.root, text="부하 전류 (A):").grid(row=5, column=2)
        self.current_entry = tk.Entry(self.root, width=10)
        self.current_entry.grid(row=5, column=3)
        self.current_entry.insert(0, "11.0")

        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=7, column=0, columnspan=2)

        self.stop_test_btn = tk.Button(self.root, text="테스트 정지", command=self.stop_test)
        self.stop_test_btn.grid(row=7, column=2, columnspan=2)

        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=8, column=0, columnspan=5)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return None
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return None

    def start_test(self):
        if not self.instrument:
            self.log("연결된 장치가 없습니다.")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        try:
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            if not voltages:
                raise ValueError("전압 시퀀스를 입력하세요.")
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            if wait_time <= 0 or sampling_interval <= 0:
                raise ValueError("대기 시간과 샘플링 간격은 0보다 커야 합니다.")
            load_current = float(self.current_entry.get())
            if load_current <= 0:
                raise ValueError("부하 전류는 0보다 커야 합니다.")
        except Exception as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return

        self.voltages = voltages
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        self.load_current = load_current

        self.open_log_file()
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def stop_test(self):
        self.is_testing = False
        self.log("테스트 중지 요청됨.")

    def open_log_file(self):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.csv_file = open(f"result_{timestamp}.csv", "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Time", "Set_V", "Measured_V", "Current", "Power", "Temperature", "Supply_v", "PowerMeter"])
        self.log("CSV 파일 생성 완료")

    def run_test_sequence(self):
        try:
            supply_label = "Power Supply"
            load_label = "Electronic Load"
            temp_label = "온도 센서"
            power_meter_label = "Power Meter"

            for v in self.voltages:
                if not self.is_testing:
                    break
                self.log(f"전압 {v}V 설정 및 테스트 시작")
                time.sleep(0.5)
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                        time.sleep(1)
                        self.write(supply_label, "AR 1")
                        time.sleep(1)
                        self.write(supply_label, f"VOLT {v}")
                        time.sleep(1)
                        self.write(supply_label, "TEST")
                        time.sleep(1)
                    except Exception as e:
                        self.log(f"Power Supply 명령 오류: {e}")

                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":MODE CC")
                        self.write(load_label, f":CURR {self.load_current}")
                        self.write(load_label, ":LOAD ON")
                    except Exception as e:
                        self.log(f"Electronic Load 명령 오류: {e}")

                start_time = time.time()
                while time.time() - start_time < self.wait_time:
                    if not self.is_testing:
                        break
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    supply_v = temperature = voltage_meas = current_meas = power_meas = power_meter_val = "N/A"
                    try:
                        if supply_label in self.instrument:
                            val = self.query(supply_label, "TDVOLT?")
                            if val is not None:
                                supply_v = float(val)
                        if temp_label in self.instrument:
                            val = self.query(temp_label, ":VAL1:VAL?")
                            if val is not None:
                                temperature = float(val)
                        if load_label in self.instrument:
                            val_v = self.query(load_label, ":MEAS:VOLT?") # 이거 필요 없을지도?
                            val_c = self.query(load_label, ":MEAS:CURR?") # 이것도
                            val_p = self.query(load_label, ":MEAS:POW?") # 이것도
                            if val_v is not None:
                                voltage_meas = float(val_v)
                            if val_c is not None:
                                current_meas = float(val_c)
                            if val_p is not None:
                                power_meas = float(val_p)
                        if power_meter_label in self.instrument:
                            val = self.query(power_meter_label, ":MEAS:POW?")
                            if val is None or val == "":
                                val = self.query(power_meter_label, ":MEAS:VOLT?")
                            if val is not None:
                                power_meter_val = float(val)
                    except Exception as e:
                        self.log(f"측정 오류: {e}")

                    self.csv_writer.writerow([timestamp, v, voltage_meas, current_meas, power_meas, temperature, supply_v, power_meter_val])
                    self.csv_file.flush()
                    self.log(f"[{timestamp}] V={voltage_meas}, I={current_meas}, P={power_meas}, T={temperature}, M={power_meter_val}")
                    time.sleep(self.sampling_interval)

                if load_label in self.instrument:
                    try:
                        self.write(load_label, ":LOAD OFF")
                    except Exception as e:
                        self.log(f"Electronic Load 종료 명령 오류: {e}")
                if supply_label in self.instrument:
                    try:
                        self.write(supply_label, "*RST")
                    except Exception as e:
                        self.log(f"Power Supply 종료 명령 오류: {e}")
                time.sleep(2)
            self.log("모든 테스트 완료")
        except Exception as e:
            self.log(f"시퀀스 실행 중 오류: {e}")
        finally:
            self.is_testing = False
            if self.csv_file:
                self.csv_file.close()
                self.log("CSV 저장 완료")
            for label, inst in self.instrument.items():
                try:
                    if self.connection_type[label] == "VISA":
                        inst.close()
                    elif self.connection_type[label] == "SERIAL":
                        inst.close()
                    elif self.connection_type[label] == "LAN":
                        inst.close()
                except Exception:
                    pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()

# 파워미터로 값 + 로드 채널 여러개 경우(전류 설정) 

# 전체적인 gui만 

In [2]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.dynamic_current_entries = {}  # {voltage: [(frame, label, entry)]}
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서", "Power Meter"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry

        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=4, sticky="ns")

        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=4, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=4, column=1)
        self.voltage_entry.insert(0, "60,90")

        generate_btn = tk.Button(self.root, text="채널 설정 생성", command=self.generate_channel_fields)
        generate_btn.grid(row=4, column=2)

        tk.Label(self.root, text="대기 시간 (s):").grid(row=5, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=5, column=1)
        self.wait_entry.insert(0, "300")

        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=6, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=6, column=1)
        self.sampling_entry.insert(0, "5")

        self.channel_frame = tk.Frame(self.root)
        self.channel_frame.grid(row=7, column=0, columnspan=5, sticky="w")

        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=8, column=0, columnspan=2)

        self.stop_test_btn = tk.Button(self.root, text="테스트 정지", command=self.stop_test)
        self.stop_test_btn.grid(row=8, column=2, columnspan=2)

        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=9, column=0, columnspan=5)

    def generate_channel_fields(self):
        for widget in self.channel_frame.winfo_children():
            widget.destroy()
        self.dynamic_current_entries.clear()

        voltages = [v.strip() for v in self.voltage_entry.get().split(",") if v.strip()]
        for v in voltages:
            section_label = tk.Label(self.channel_frame, text=f"[{v}V] 채널 설정:")
            section_label.pack(anchor="w")

            # 가로 프레임 생성
            row_frame = tk.Frame(self.channel_frame)
            row_frame.pack(anchor="w", pady=2)

            add_btn = tk.Button(row_frame, text=f"{v}V 채널 +", command=lambda vv=v: self.add_channel_row(vv))
            add_btn.pack(side="left")

            self.dynamic_current_entries[v] = {
                "frame": row_frame,
                "entries": []  # list of (label, entry, button)
            }

    def add_channel_row(self, voltage):
        container = self.dynamic_current_entries[voltage]
        row_frame = container["frame"]
        ch_num = len(container["entries"]) + 1

        ch_label = tk.Label(row_frame, text=f"CH{ch_num}:")
        ch_label.pack(side="left", padx=2)

        current_entry = tk.Entry(row_frame, width=6)
        current_entry.pack(side="left", padx=2)

        del_btn = tk.Button(row_frame, text="-", command=lambda: self.remove_channel_entry(voltage, ch_label, current_entry, del_btn))
        del_btn.pack(side="left", padx=2)

        container["entries"].append((ch_label, current_entry, del_btn))
    def remove_channel_entry(self, voltage, label, entry, button):
        container = self.dynamic_current_entries[voltage]
        label.destroy()
        entry.destroy()
        button.destroy()
        container["entries"] = [triple for triple in container["entries"] if triple[0] != label]
        
    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return None
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return None

    def start_test(self):
        self.log("테스트 시작 로직을 여기에 구현하세요. self.dynamic_current_entries에서 설정값을 가져오세요.")

    def stop_test(self):
        self.is_testing = False
        self.log("테스트 중지 요청됨.")

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()


# 전체 GUI + 기능

In [6]:
import tkinter as tk
from tkinter import messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.dynamic_current_entries = {}  # {voltage: [(frame, label, entry)]}
        self.build_gui()

    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서", "Power Meter"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry

        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=4, sticky="ns")

        tk.Label(self.root, text="전압 시퀀스 설정 (V):").grid(row=4, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=4, column=1)
        self.voltage_entry.insert(0, "60,90")

        generate_btn = tk.Button(self.root, text="채널 설정 생성", command=self.generate_channel_fields)
        generate_btn.grid(row=4, column=2)

        tk.Label(self.root, text="대기 시간 (s):").grid(row=5, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=5, column=1)
        self.wait_entry.insert(0, "300")

        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=6, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=6, column=1)
        self.sampling_entry.insert(0, "5")

        self.channel_frame = tk.Frame(self.root)
        self.channel_frame.grid(row=7, column=0, columnspan=5, sticky="w")

        self.start_test_btn = tk.Button(self.root, text="테스트 시작", command=self.start_test)
        self.start_test_btn.grid(row=8, column=0, columnspan=2)

        self.stop_test_btn = tk.Button(self.root, text="테스트 정지", command=self.stop_test)
        self.stop_test_btn.grid(row=8, column=2, columnspan=2)

        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=9, column=0, columnspan=5)

    def generate_channel_fields(self):
        for widget in self.channel_frame.winfo_children():
            widget.destroy()
        self.dynamic_current_entries.clear()

        voltages = [v.strip() for v in self.voltage_entry.get().split(",") if v.strip()]
        for v in voltages:
            section_label = tk.Label(self.channel_frame, text=f"[{v}V] 채널 설정:")
            section_label.pack(anchor="w")

            row_frame = tk.Frame(self.channel_frame)
            row_frame.pack(anchor="w", pady=2)

            add_btn = tk.Button(row_frame, text=f"{v}V 채널 +", command=lambda vv=v: self.add_channel_row(vv))
            add_btn.pack(side="left")

            self.dynamic_current_entries[v] = {
                "frame": row_frame,
                "entries": []
            }

    def add_channel_row(self, voltage):
        container = self.dynamic_current_entries[voltage]
        row_frame = container["frame"]
        ch_num = len(container["entries"]) + 1

        ch_label = tk.Label(row_frame, text=f"CH{ch_num}:")
        ch_label.pack(side="left", padx=2)

        current_entry = tk.Entry(row_frame, width=6)
        current_entry.pack(side="left", padx=2)

        del_btn = tk.Button(row_frame, text="-", command=lambda: self.remove_channel_entry(voltage, ch_label, current_entry, del_btn))
        del_btn.pack(side="left", padx=2)

        container["entries"].append((ch_label, current_entry, del_btn))

    def remove_channel_entry(self, voltage, label, entry, button):
        container = self.dynamic_current_entries[voltage]
        label.destroy()
        entry.destroy()
        button.destroy()
        container["entries"] = [triple for triple in container["entries"] if triple[0] != label]

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def start_test(self):
        if not self.instrument:
            self.log("연결된 장치가 없습니다.")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        try:
            voltages = [v.strip() for v in self.voltage_entry.get().split(",") if v.strip()]
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
        except Exception as e:
            self.log(f"입력 오류: {e}")
            return

        load_label = "Electronic Load"
        supply_label = "Power Supply"

        timestamp = time.strftime("%Y%m%d_%H%M%S")
        self.csv_file = open(f"result_{timestamp}.csv", "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        self.csv_writer.writerow(["Time", "Voltage", "Channel", "Current"])

        self.is_testing = True

        def test_sequence():
            for v in voltages:
                self.write(supply_label, f"VOLT {v}")
                time.sleep(1)
                channel_data = self.dynamic_current_entries.get(v, {}).get("entries", [])
                for idx, (_, entry, _) in enumerate(channel_data):
                    try:
                        current = float(entry.get())
                        self.write(load_label, f":CHAN {idx+1}")
                        self.write(load_label, ":MODE CC")
                        self.write(load_label, f":CURR {current}")
                        self.write(load_label, ":LOAD ON")
                        self.log(f"{v}V - CH{idx+1} 부하 설정: {current}A")
                    except Exception as e:
                        self.log(f"CH{idx+1} 전류 설정 오류: {e}")

                start = time.time()
                while time.time() - start < wait_time:
                    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    for idx, (_, entry, _) in enumerate(channel_data):
                        try:
                            self.write(load_label, f":CHAN {idx+1}")
                            current_meas = self.query(load_label, ":MEAS:CURR?")
                            self.csv_writer.writerow([now, v, f"CH{idx+1}", current_meas])
                            self.csv_file.flush()
                        except Exception as e:
                            self.log(f"측정 오류 CH{idx+1}: {e}")
                    time.sleep(sampling_interval)

                for idx, _ in enumerate(channel_data):
                    self.write(load_label, f":CHAN {idx+1}")
                    self.write(load_label, ":LOAD OFF")
                time.sleep(2)

            self.is_testing = False
            self.log("테스트 완료")
            self.csv_file.close()

        self.test_thread = threading.Thread(target=test_sequence, daemon=True)
        self.test_thread.start()

    def stop_test(self):
        self.is_testing = False
        self.log("테스트 중지 요청됨.")

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()

Exception in thread Thread-7 (test_sequence):
Traceback (most recent call last):
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_21676\2205604604.py", line 208, in test_sequence
AttributeError: 'PowerAnalyzerGUI' object has no attribute 'query'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lenovo\anaconda3\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\lenovo\anaconda3\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_21676\2205604604.py", line 212, in test_sequence
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_21676\2205604604.py", line 116, in log
  File "c:\Users\lenovo\anaconda3\Lib\tkinter\__init__.py", line 872, in after
    name = self._register(callit)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\lenovo\anaconda3\Lib\tkinter\__init__.py", line 1584, in _register
    s

# 다시 모든 기능 + gui 

In [13]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pyvisa
import serial
import socket
import threading
import csv
import time
from datetime import datetime

class PowerAnalyzerGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("통합 테스트 시스템")
        self.connection_type = {}
        self.instrument = {}
        self.csv_file = None
        self.csv_writer = None
        self.is_testing = False
        self.test_thread = None
        self.rm = pyvisa.ResourceManager()
        self.save_folder = None
        self.dl_currents = {}
        self.build_gui()
    def build_gui(self):
        labels = ["Power Supply", "Electronic Load", "온도 센서", "Power Meter"]
        self.device_entries = {}
        for i, label in enumerate(labels):
            tk.Label(self.root, text=label).grid(row=i, column=0)
            entry = tk.Entry(self.root, width=40)
            entry.grid(row=i, column=1, columnspan=3)
            self.device_entries[label] = entry

        self.connect_btn = tk.Button(self.root, text="장비 연결", command=self.connect_all)
        self.connect_btn.grid(row=0, column=4, rowspan=4, sticky="ns")

        # 전압 시퀀스, 주파수, 대기 시간, 샘플링 간격
        tk.Label(self.root, text="전압 시퀀스 (V):").grid(row=4, column=0)
        self.voltage_entry = tk.Entry(self.root, width=30)
        self.voltage_entry.grid(row=4, column=1)
        self.voltage_entry.insert(0, "90,264")

        tk.Label(self.root, text="주파수 (Hz):").grid(row=5, column=0)
        self.freq_entry = tk.Entry(self.root, width=10)
        self.freq_entry.grid(row=5, column=1)
        self.freq_entry.insert(0, "50")

        tk.Label(self.root, text="대기 시간 (s):").grid(row=6, column=0)
        self.wait_entry = tk.Entry(self.root, width=10)
        self.wait_entry.grid(row=6, column=1)
        self.wait_entry.insert(0, "300")

        tk.Label(self.root, text="샘플링 간격 (s):").grid(row=7, column=0)
        self.sampling_entry = tk.Entry(self.root, width=10)
        self.sampling_entry.grid(row=7, column=1)
        self.sampling_entry.insert(0, "5")

        # DL 다채널 전류 설정
        tk.Label(self.root, text="DL 채널별 전류 (A):").grid(row=8, column=0, sticky="w")
        self.dl_current_entries = {}
        for ch in range(1, 6):
            tk.Label(self.root, text=f"{ch}CH").grid(row=8, column=ch)
            entry = tk.Entry(self.root, width=6)
            entry.grid(row=9, column=ch)
            entry.insert(0, "5.0")
            self.dl_current_entries[ch] = entry

        # HR 온도센서 채널 설정
        tk.Label(self.root, text="온도센서 채널:").grid(row=10, column=0)
        self.hr_channel_entry = tk.Entry(self.root, width=5)
        self.hr_channel_entry.grid(row=10, column=1)
        self.hr_channel_entry.insert(0, "1")

        # Power Meter 표시 (측정된 값 실시간 보기용)
        self.pm_v_label = tk.Label(self.root, text="PM V: ---")
        self.pm_v_label.grid(row=11, column=0)
        self.pm_a_label = tk.Label(self.root, text="PM A: ---")
        self.pm_a_label.grid(row=11, column=1)
        self.pm_hz_label = tk.Label(self.root, text="PM Hz: ---")
        self.pm_hz_label.grid(row=11, column=2)

        # 폴더 지정 버튼
        self.folder_btn = tk.Button(self.root, text="폴더 지정", command=self.select_folder)
        self.folder_btn.grid(row=12, column=0)

        # 시작 / 중지 버튼
        self.start_test_btn = tk.Button(self.root, text="시작", command=self.start_test)
        self.start_test_btn.grid(row=12, column=1)

        self.stop_test_btn = tk.Button(self.root, text="중지", command=self.stop_test)
        self.stop_test_btn.grid(row=12, column=2)

        self.log_box = tk.Text(self.root, height=15, width=100)
        self.log_box.grid(row=13, column=0, columnspan=5)

    def log(self, text):
        def append_log():
            timestamp = time.strftime("%H:%M:%S")
            self.log_box.insert(tk.END, f"[{timestamp}] {text}\n")
            self.log_box.see(tk.END)
        self.root.after(0, append_log)

    def connect_all(self):
        for label, entry in self.device_entries.items():
            address = entry.get().strip()
            if not address:
                continue
            try:
                if address.upper().startswith("USB") or "::" in address:
                    inst = self.rm.open_resource(address)
                    inst.timeout = 3000
                    inst.query("*IDN?")
                    self.instrument[label] = inst
                    self.connection_type[label] = "VISA"
                    self.log(f"{label} 연결 성공 (VISA)")
                elif address.upper().startswith("COM"):
                    ser = serial.Serial(address, baudrate=9600, timeout=3)
                    self.instrument[label] = ser
                    self.connection_type[label] = "SERIAL"
                    self.log(f"{label} 연결 성공 (SERIAL)")
                elif ":" in address:
                    ip, port = address.split(":")
                    sock = socket.create_connection((ip, int(port)), timeout=3)
                    self.instrument[label] = sock
                    self.connection_type[label] = "LAN"
                    self.log(f"{label} 연결 성공 (LAN)")
                else:
                    raise ValueError("주소 형식을 인식할 수 없음.")
            except Exception as e:
                self.log(f"{label} 연결 실패: {e}")

    def write(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                inst.write(command)
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
        except Exception as e:
            self.log(f"{label} 명령 전송 오류: {e}")

    def query(self, label, command):
        try:
            if label not in self.connection_type or label not in self.instrument:
                return None
            conn_type = self.connection_type[label]
            inst = self.instrument[label]
            if conn_type == "VISA":
                return inst.query(command).strip()
            elif conn_type == "SERIAL":
                inst.write((command + "\n").encode())
                time.sleep(0.3)
                return inst.readline().decode().strip()
            elif conn_type == "LAN":
                inst.sendall((command + "\n").encode())
                return inst.recv(1024).decode().strip()
        except Exception as e:
            self.log(f"{label} 쿼리 오류: {e}")
            return None

    def select_folder(self):
        self.save_folder = filedialog.askdirectory()
        if self.save_folder:
            self.log(f"저장 폴더 지정됨: {self.save_folder}")

    def start_test(self):
        if not self.instrument:
            self.log("연결된 장치가 없습니다.")
            messagebox.showerror("연결 오류", "연결된 장치가 없습니다.")
            return

        try:
            voltages = [float(v.strip()) for v in self.voltage_entry.get().split(",") if v.strip()]
            frequency = float(self.freq_entry.get())
            wait_time = float(self.wait_entry.get())
            sampling_interval = float(self.sampling_entry.get())
            hr_channel = self.hr_channel_entry.get().strip()

            dl_currents = {}
            for ch in range(1, 6):
                val = self.dl_current_entries[ch].get().strip()
                if val:
                    dl_currents[ch] = float(val)

            if not self.save_folder:
                messagebox.showwarning("경고", "저장 폴더가 지정되지 않았습니다.")
                return

        except Exception as e:
            self.log(f"입력값 오류: {e}")
            messagebox.showerror("입력값 오류", str(e))
            return

        self.voltages = voltages
        self.frequency = frequency
        self.wait_time = wait_time
        self.sampling_interval = sampling_interval
        self.hr_channel = hr_channel
        self.dl_currents = dl_currents

        self.open_log_file()
        self.is_testing = True
        self.test_thread = threading.Thread(target=self.run_test_sequence, daemon=True)
        self.test_thread.start()

    def stop_test(self):
        self.is_testing = False
        self.log("테스트 중지 요청됨.")

    def open_log_file(self):
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(self.save_folder, f"result_{timestamp}.csv")
        self.csv_file = open(filename, "w", newline="")
        self.csv_writer = csv.writer(self.csv_file)
        header = ["Time", "Set_V", "Set_Hz"]
        for ch in range(1, 6):
            header.extend([f"DL{ch}_V", f"DL{ch}_A"])
        header.extend(["PM_V", "PM_A", "PM_Hz", "Temperature"])
        self.csv_writer.writerow(header)
        self.log(f"CSV 파일 생성 완료: {filename}")

    def run_test_sequence(self):
        try:
            ps_label = "Power Supply"
            dl_label = "Electronic Load"
            hr_label = "온도 센서"
            pm_label = "Power Meter"

            for v in self.voltages:
                if not self.is_testing:
                    break
                self.log(f"전압 {v}V, 주파수 {self.frequency}Hz 설정 중")
                time.sleep(0.5)
                if ps_label in self.instrument:
                    try:
                        self.write(ps_label, "*RST")
                        time.sleep(1)
                        self.write(ps_label, f"VOLT {v}")
                        self.write(ps_label, f"FREQ {self.frequency}")
                        self.write(ps_label, "TEST")
                        time.sleep(1)
                    except Exception as e:
                        self.log(f"Power Supply 설정 오류: {e}")

                if dl_label in self.instrument:
                    try:
                        self.write(dl_label, ":MODE CC")
                        for ch, current in self.dl_currents.items():
                            self.write(dl_label, f":CHAN {ch}")
                            self.write(dl_label, f":CURR {current}")
                            self.write(dl_label, ":LOAD ON")
                    except Exception as e:
                        self.log(f"Electronic Load 설정 오류: {e}")

                start_time = time.time()
                while time.time() - start_time < self.wait_time:
                    if not self.is_testing:
                        break
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    row = [timestamp, v, self.frequency]
                    try:
                        if dl_label in self.instrument:
                            for ch in range(1, 6):
                                self.write(dl_label, f":CHAN {ch}")
                                val_v = self.query(dl_label, ":MEAS:VOLT?")
                                val_a = self.query(dl_label, ":MEAS:CURR?")
                                row.append(float(val_v) if val_v else "N/A")
                                row.append(float(val_a) if val_a else "N/A")
                        else:
                            row.extend(["N/A", "N/A"] * 5)

                        pm_v = self.query(pm_label, ":MEAS:VOLT?") if pm_label in self.instrument else "N/A"
                        pm_a = self.query(pm_label, ":MEAS:CURR?") if pm_label in self.instrument else "N/A"
                        pm_hz = self.query(pm_label, ":MEAS:FREQ?") if pm_label in self.instrument else "N/A"

                        self.pm_v_label.config(text=f"PM V: {pm_v}")
                        self.pm_a_label.config(text=f"PM A: {pm_a}")
                        self.pm_hz_label.config(text=f"PM Hz: {pm_hz}")

                        row.append(float(pm_v) if pm_v else "N/A")
                        row.append(float(pm_a) if pm_a else "N/A")
                        row.append(float(pm_hz) if pm_hz else "N/A")

                        hr_val = self.query(hr_label, f":VAL{self.hr_channel}:VAL?") if hr_label in self.instrument else "N/A"
                        row.append(float(hr_val) if hr_val else "N/A")

                    except Exception as e:
                        self.log(f"측정 오류: {e}")
                        row.extend(["ERR"] * 13)

                    self.csv_writer.writerow(row)
                    self.csv_file.flush()
                    self.log(f"[{timestamp}] 측정 완료 → {row}")
                    time.sleep(self.sampling_interval)

                if dl_label in self.instrument:
                    for ch in range(1, 6):
                        self.write(dl_label, f":CHAN {ch}")
                        self.write(dl_label, ":LOAD OFF")
                if ps_label in self.instrument:
                    self.write(ps_label, "*RST")
                time.sleep(1)

            self.log("모든 테스트 완료")

        except Exception as e:
            self.log(f"테스트 시퀀스 오류: {e}")
        finally:
            self.is_testing = False
            if self.csv_file:
                self.csv_file.close()
                self.log("CSV 저장 완료")
            for label, inst in self.instrument.items():
                try:
                    if self.connection_type[label] == "VISA":
                        inst.close()
                    elif self.connection_type[label] == "SERIAL":
                        inst.close()
                    elif self.connection_type[label] == "LAN":
                        inst.close()
                except Exception:
                    pass

if __name__ == "__main__":
    root = tk.Tk()
    app = PowerAnalyzerGUI(root)
    root.mainloop()
